# Solving Ehrenfest Diffusion with Markov Chains

This excersize covers the problem put forward by the famous physicist and one of the fathers of statistical mechanics. We will learn how to formulate a problem in terms of specific states and calculate probabilities based on the Markov property. We will in particular:
1. Compute probabilities based on matrix operations.
2. Compare the computed results with simple simulations.

The exercise is made to play around with a simple thought experiment and combine calculations with direct simulations. This is not curriculum, but can turn in valuable for your future careers. 

### References:
- Lecture on Markov Chains

### Authors: 
- Mathias L. Heltberg (Niels Bohr Institute)

### Date:    
- 29-12-2021 (latest update)

***

As always we will include a collection of libraries (Standard)

In [ ]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
from numpy.linalg import matrix_power                  # This is to raise matrices to powers
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Module to see files and folders in directories
from scipy import stats

Naturally we will here need a random generator:

In [ ]:
r = np.random # Random generator
r.seed(42)    # Set a random seed (but a fixed one)

And now for the actual problem: We have two baskets, a blue and a red. We have 10 balls, and each turn we pick a random ball and move it from the basket it is in, over to the other basket.

**We want to compute the probability that there are 7 balls in the blue basket after 20 turns, given that 3 balls are in the blue container in the beginning.**

This assignment is tricky, since the probability must be affected by the number of balls after 19 turns, and this is affected by the situation after 18 turns etc.

Therefore we start by simulating the situation:

In [ ]:
NB = 10; ### This is the number of balls - note the number of states is NB+1. Do you understand why?
Initial_Place = np.zeros((NB,), dtype=int)
Initial_Place[3:] = 1; ### Make initial distribution
print(Initial_Place)

Now we have the initial setting in place. We next simulate the 20 turns:

In [ ]:
ntest = 50 ### Number of times we perform the experiment - this can be changed
nturns = 20 ### The 20 turns we pick the balls
H20 = np.zeros(ntest)
for itest in range(ntest):
    Place = Initial_Place.copy()
    Nballs = np.zeros(nturns+1) ### Here we count the number of balls in the blue basket (i.e. basket #0)
    Nballs[0] = np.sum(Place==0)
    for iturns in range(nturns):
        id = np.random.randint(0,10) ### Generates an integer between 0 and 7
        if (Place[id] == 0):
            Place[id] = 1
        else:
            Place[id] = 0
        Nballs[iturns +1] = np.sum(Place==0)

    
    if (itest == 0):
        fig = plt.figure(figsize=[7,5])
        ax = plt.subplot(111)
        ax.plot(np.arange(0,nturns+1,1), Nballs,'-x')
        ax.set_xlabel('Turn #')
        ax.set_ylabel('State')
        ax.set_title('Evolution of state')

        # set the limits
        ax.set_xlim(0, 20)
        ax.set_ylim(0, 10)

        # set the grid on
        ax.grid('on')
    H20[itest] = np.sum(Place==0)
    


So far we have plotted one realization of the ball swapping experiment. Lets us plot the distribution after 20 turns...

In [ ]:
plt.figure(figsize=(10,6), tight_layout=True)
bins = np.arange(0,11,1)
# matplotlib
plt.hist(H20, bins=bins, color=sns.color_palette('Set2')[2], linewidth=4)
plt.title('State after 20 turns')
plt.xlabel('State #')
plt.ylabel('Occurrence')
print(H20)


So far for the simulations. We now go to the derivations based on the Markov Chain formulation. We construct the matrix of jump probabilities. It can be constructed by:

In [ ]:
M = np.zeros((NB+1,NB+1))
for i in range(NB+1):
    if (i==0):
        M[i,i+1] = (NB-i)/NB
    elif (i==NB):
        M[i,i-1] = i/NB
    else:
        M[i,i-1] = i/NB
        M[i,i+1] = (NB-i)/NB

print(M)

Based on this matrix, we want to compute the probabilities of being in the different states. This is what we do here:

In [ ]:
V1 = np.zeros((1,NB+1)); V1[0,3] = 1; ### Note V1 represents the initial probability of states (all is in state 3)
V2 = np.zeros(NB+1); V2[9] = 1 ### Note V2 represents the outcome probability (Here we are interested in state 9)
print('Probability of initial state is:',V1)

PAfter = np.matmul(V1,matrix_power(np.ones((NB+1,NB+1)),1)) #### Insert your calculations here!!!##########
P9 = np.matmul(PAfter,np.zeros(NB+1)) #### Insert your calculations here!!! #############
print('The probability array after n swappings is: ',PAfter)

# Questions:

This exercise is made to understand and simulate a simple thought experiment, where the calculations can easily be very tedious. The questions here aims at understanding the results and playing with the system:

1) Start by looking at the matrix M. Do you understand why it has the shape it does? What is the probability to have 9 balls in the blue basked (basket #0) after 21 turns? Why is this?

2) Rewrite the system so at each turn there is a 50% chance of nothing happens (ie no ball is moved). How does this affect the probabilities? Can all states be reached after 20 turns?

3) Change the number of turns to something much larger (try 1000). What value is the most probable to reach? Does this make sense?

### Advanced question:

4) Assume that the state #10 (where all balls are blue) is an absorbing state, so nothing changes after this state is reached. What is the average number of turns to reach this? Can you find a PDF that fits the distribution of first passage times?

5) Assume that the initial position of the balls is not always three blue, but instead a poissonian distribution with mean=1. How does this change the distribution to reach state 10? Can you fit this distribution?

# Learning points

This is simple introduction of the application of Markov Chains and their use in calculations on simple probability questions.

What is important in this, is that all probabilities at a specific state, depends completely on the last state etc. This makes all calculations iterative and this is almost impossible from "standard" frequentist approaches. The construction of a Transfer matrix simplifies such calculations enormously and it is hoped you will appreciate this.

From this exercise, you should have learned:

1) How to calculate probabilities in a problem, where measurements are not independent and the (n+1)'th state is based on the n'th state. 

2) How to construct a transfer matrix based on a simple setup.

3) Understand why an equlibrium state is reached and how one can use a probability distribution in the first state to calculate a probability in the n'th state.